**import required libraries**


In [31]:
import pandas as pd
import json
from pathlib import Path

**load required files into pandas dataframes**

In [32]:
data = Path("../data")

users_df = pd.read_csv(data/"users_2017.tsv", sep='\t', parse_dates=["signup_date", "start_date"])
recordings_df = pd.read_csv(data/"recordings_2017.tsv", sep='\t', parse_dates=["Date_Time"])

**convert column names to lower case for consistency**

In [33]:
users_df.columns = users_df.columns.str.lower()
recordings_df.columns = recordings_df.columns.str.lower()

**break down recording summary JSON into separate columns**

In [34]:
recordings_df["recording_summary"] = recordings_df["recording_summary"].fillna('{}').astype(str) # to handle nulls in recording_summary
recording_summary_df = recordings_df["recording_summary"].apply(json.loads).apply(pd.Series) # parse JSON and break down it into separate columns
recordings_df = pd.concat([recordings_df.drop(columns=["recording_summary"]), recording_summary_df], axis=1) # merge into recordings_df
recordings_df["updatedAt"] = pd.to_datetime(recordings_df["updatedAt"]) # convert updatedAt to datetime

**clean up column names in both dataframes**

In [35]:
users_df = users_df.rename(columns={"signup_date": "user_signup_date", "start_date": "user_start_date"})
recordings_df = recordings_df.rename(columns={"date_time": "recording_date_time", "timeTotal": "time_total", "updatedAt": "updated_at", "timeMoving": "time_moving", "paceAverage": "pace_average", "speedAverage": "speed_average", "distanceTotal": "distance_total", "elevationGain": "evelation_gain", "elevationLoss": "elevation_loss"})

**Get the second recording for each user**

In [36]:
recordings_df = recordings_df.sort_values(by=["pseudo_user_id", "recording_date_time"])
second_recording_df = recordings_df.groupby("pseudo_user_id").nth(1).reset_index()

**join second_recording_df with users_df**

In [37]:
users_second_recording_df = pd.merge(users_df, second_recording_df, on="pseudo_user_id", how="inner")

**find the difference between user's signup date and their second recording in hours**

In [38]:
users_second_recording_df["hours_till_second_recording"] = (users_second_recording_df["recording_date_time"] - users_second_recording_df["user_signup_date"]).dt.total_seconds()/3600

**filter out negative values the same way as it is done with the first recordings**

In [39]:
negative_hours_df = users_second_recording_df[users_second_recording_df['hours_till_second_recording'] < 0] # 59 records

In [40]:
users_second_recording_df = users_second_recording_df[users_second_recording_df['hours_till_second_recording'] >= 0]

**export final df to CSV**

In [41]:
output_dir = Path("../output")
users_second_recording_df.to_csv(output_dir/"users_second_recording_2017.csv", index=False)